In [60]:
import pandas as pd

In [61]:
country = "Netherlands"
year = 2017
election_type = "camera"
aggregation = "district"

In [62]:
df = pd.read_csv(f"data_output/{country}/{year}_{election_type}_{aggregation}.csv.gz")
print(df.shape)
df = df[df["candidate"].str.strip() != ""]
print(df.shape)
df = df[~df["candidate"].isna()]
print(df.shape)

df.head()

(13796, 6)
(13796, 6)
(13796, 6)


,candidate,elected,value,party,party_id,district_id
0,M. (Mark) Rutte,False,53852,VVD,586934,686937
1,M.F. (Mark) Strolenberg,False,3561,VVD,586934,686937
2,T. (Tamara) van Ark,False,1944,VVD,586934,686937
3,R.J. (Roelien) Kamminga,False,1191,VVD,586934,686937
4,D. (Dilan) Yeşilgöz-Zegerius,False,931,VVD,586934,686937


In [63]:
df = df.groupby(["candidate", "party_id", "district_id"]).agg({"value": "sum"}).reset_index()

In [64]:
coalition = df.groupby(["district_id", "party_id"]).agg({"value": "sum", "candidate": "nunique"}).rename(columns={"candidate": "Q", "value": "N"}).reset_index()
coalition.head()

,district_id,party_id,N,Q
0,686935,586934,53539,80
1,686935,586935,36595,50
2,686935,586936,31758,73
3,686935,586937,55182,80
4,686935,586938,23333,50


In [65]:
data = pd.merge(df, coalition, on=["district_id", "party_id"])
data["vQ/N"] = data["value"] * data["Q"] / data["N"]
data.head()

,candidate,party_id,district_id,value,N,Q,vQ/N
0,A. Bakir,586946,686935,11,18312,50,0.030035
1,A. Kuijer,586946,686935,4,18312,50,0.010922
2,A. de Kok,586946,686935,126,18312,50,0.344037
3,A.H.J.W. van Schijndel,586946,686935,1,18312,50,0.002730
4,A.J.H. van Essen,586946,686935,1,18312,50,0.002730


In [66]:
data.to_csv(f"data_universality/{country}_{year}_{election_type}.csv", index=False)